In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Madaar129c\\Desktop\\chicken-disease-classification\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\Madaar129c\\Desktop\\chicken-disease-classification'

Creating Custom Entity

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

Adding Configurations for Entity

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
            self, 
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config

Creating Component

In [8]:
import os
import urllib.request as request
import zipfile
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f'{filename} downloaded with the following info: \n{headers}')
        else:
            logger.info(f'file already exists of size: {get_size(Path(self.config.local_data_file))}')

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

Creating Pipeline

In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-03-28 03:11:58,367: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-03-28 03:11:58,369: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-28 03:11:58,370: INFO: common: created directory at: artifacts]
[2024-03-28 03:11:58,371: INFO: common: created directory at: artifacts/data_ingestion]
[2024-03-28 03:12:05,480: INFO: 2746037919: artifacts/data_ingestion/dataset.zip downloaded with the following info: 
Connection: close
Content-Length: 11595077
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "23ae950c456ac0156d4b66346c50c23c0b115d8db43114c53a66a453b0aae906"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 6EEE:362EF2:14F4925:15C0393:660499AE
Accept-Ranges: bytes
Date: Wed, 27 Mar 2024 22:11:59 GMT
Via: 1.1 varnish
X-Served-By: cache-fra-eddf8230